# Redis Clustering Demo

## Constants
Here some basic constants which we might need for demonstration purposes:

In [137]:
DB_ENDPOINT='redis-16379.internal.cluster.ubuntu-docker.org'
DB_PORT=16379
CLUSTER_NODE_1='172.1.0.2'
CLUSTER_NODE_2='172.1.0.4'
ADMIN='admin@ubuntu-docker.org'
ADMIN_PWD='redis'

## Imports

In [72]:
import redis
from rediscluster import RedisCluster
import socket
import requests
from requests.auth import HTTPDigestAuth
import json
import uuid

## Standard client example with ‘redis-py’

Redis Enterprise allows to connect to a clustered database via a single endpoint name

1. Start with a sharded DB which uses a single Proxy
2. Show how the endpoint is resolved
3. Change the database's endpoint binding to use multiple proxies
4. Show how the endpoint is resolved

In [90]:
r = redis.Redis(
    host=DB_ENDPOINT,
    port=DB_PORT,
    decode_responses=True)

In [91]:
r.set('hello', 'world')
r.get('hello')

'world'

In [92]:
## Single Proxy
print(socket.gethostbyname(DB_ENDPOINT)) 

172.1.0.3


In [93]:
## Multi-Proxy with built-in DNS balancing
for i in range(6):
    print(socket.gethostbyname(DB_ENDPOINT))

172.1.0.2
172.1.0.3
172.1.0.4
172.1.0.4
172.1.0.2
172.1.0.3


## Show a database’s sharding expression

Let's take a look at the database's sharding expression:

In [94]:
url = "http://{0}:{1}@{2}:8080/v1/bdbs".format(ADMIN, ADMIN_PWD,CLUSTER_NODE_1)

print(url)

resp = requests.get(url)
resp.content
if(resp.ok):
    data = json.loads(resp.content)
    print(data[0]['shard_key_regex'])
    

http://admin@ubuntu-docker.org:redis@172.1.0.2:8080/v1/bdbs
[{'regex': '.*\\{(?<tag>.*)\\}.*'}, {'regex': '(?<tag>.*)'}]


## Explain MULTI EXEC on a clustered database

In [100]:
# This works
r.flushdb();
## Prep
r.hmset('usr:nosqlgeek', {'name':'David Maier', 'status':'passive' })

def print_user(id):
    print("session = {0}".format(r.get('sn:{0}'.format(id))))
    print("user = {0}".format(r.hgetall('usr:{0}'.format(id))))
    print("visists = {0}".format(r.get('cnt:lgn:{0}'.format(id))))

## Update
r.set('sn:nosqlgeek', uuid.uuid1())
r.hset('usr:nosqlgeek', 'status', 'active')
r.incr('cnt:lgn:nosqlgeek')
print_user('nosqlgeek')

session = 73a021be-53f5-11e9-a224-0242ac0100dd
user = {'name': 'David Maier', 'status': 'active'}
visists = 1


In [101]:
# This is not intended to work in a clustered database as 

## Prep
### redis-py is using pipelining together with multi-exec, just fixing the name for demo purposes
r.multi=r.pipeline
r.hset('usr:nosqlgeek', 'status', 'passive')
r.delete('sn:nosqlgeek')

## Update
try:
    p=r.multi()
    p.set('sn:nosqlgeek', uuid.uuid1())
    p.hset('usr:nosqlgeek', 'status', 'active')
    p.incr('cnt:lgn:nosqlgeek')
    res=p.execute()
except Exception as e:
    print(e)

# Didn't change anything
print_user('nosqlgeek')

Command # 2 (HSET usr:nosqlgeek status active) of pipeline caused error: CROSSSLOT Keys in request don't hash to the same slot (command='HSET', key='usr:nosqlgeek') within 'MULTI'
session = None
user = {'name': 'David Maier', 'status': 'passive'}
visists = 1


In [104]:
# This works
r.flushdb();

## Prep
r.hmset('usr:{nosqlgeek}', {'name':'David Maier', 'status':'passive' })

## Update
def user_login():
    try:
        p=r.multi()
        p.set('sn:{nosqlgeek}', uuid.uuid1())
        p.hset('usr:{nosqlgeek}', 'status', 'active')
        p.incr('cnt:lgn:{nosqlgeek}')
        res=p.execute()
    except Exception as e:
        print(e)

user_login()
user_login()
print_user('{nosqlgeek}')

session = c9de375a-53f5-11e9-a224-0242ac0100dd
user = {'name': 'David Maier', 'status': 'active'}
visists = 2


## Explain LUA scripts on a clustered database

In [110]:
f="redis.call('set',KEYS[1],'c9de375a-53f5-11e9-a224-0242ac0100dd'); return redis.call('incr',KEYS[2])"

try:
    r.eval(f,2,'sn:nosqlgeek', 'cnt:lgn:nosqlgeek')
except Exception as e:
    print(e)

CROSSSLOT Keys in request don't hash to the same slot (command='EVAL', key='cnt:lgn:nosqlgeek')


In [123]:
try:
    res = r.eval(f,2,'sn:{nosqlgeek}', 'cnt:lgn:{nosqlgeek}')
    print(res)
except Exception as e:
    print(e)

5


## Sentinel Discovery

In [135]:
s = redis.Redis(
    host=CLUSTER_NODE_1,
    port=8001,
    decode_responses=True)
print('masters = {0}'.format(s.sentinel_masters()))

p = s.pubsub()
p.subscribe('+switch-master')

print('message = {0}'.format(p.get_message()));

masters = {'demo@internal': {'name': 'demo@internal', 'ip': '172.1.0.2', 'port': 16379, 'flags': 'master', 'num-other-sentinels': 0, 'is_master': True, 'is_slave': False, 'is_sdown': False, 'is_odown': False, 'is_sentinel': False, 'is_disconnected': False, 'is_master_down': False}, 'demo': {'name': 'demo', 'ip': '172.1.0.2', 'port': 16379, 'flags': 'master', 'num-other-sentinels': 0, 'is_master': True, 'is_slave': False, 'is_sdown': False, 'is_odown': False, 'is_sentinel': False, 'is_disconnected': False, 'is_master_down': False}}
message = {'type': 'subscribe', 'pattern': None, 'channel': '+switch-master', 'data': 1}


## OSS Cluster client example with 'redis-py-cluster'

1. Create a OSS Cluster DB in Redis Enterprise
2. Show and explain CLUSTER SLOT
3. Explain '-c' of redis-cli
4. Show some simple commands
5. Show a command without a key argument

In [140]:
startup_nodes = [{"host": CLUSTER_NODE_1, "port": str(DB_PORT)}, {"host": CLUSTER_NODE_2, "port": str(DB_PORT)}]
rc = RedisCluster(startup_nodes=startup_nodes, decode_responses=True)


In [142]:
rc.execute_command('CLUSTER SLOTS')

{(0, 8191): {'master': ('172.1.0.2', 16379), 'slaves': []},
 (8192, 16383): {'master': ('172.1.0.3', 16379), 'slaves': []}}

In [141]:
for i in range(100):
    rc.set("hello:{0}".format(i), "world:{0}".format(i))

rc.dbsize()

{'172.1.0.2:16379': 52, '172.1.0.3:16379': 48}